In [8]:
import cv2
import numpy as np
import os, time, random, glob
import matplotlib.pyplot as plt
import time
from skimage.feature import hog
from skimage.transform import pyramid_gaussian 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported")

DATA_DIR = '../data'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train', 'images')
TRAIN_LBL_DIR = os.path.join(DATA_DIR, 'train', 'labels')
VAL_IMG_DIR = os.path.join(DATA_DIR, 'valid', 'images')
VAL_LBL_DIR = os.path.join(DATA_DIR, 'valid', 'labels')

Libraries imported


In [9]:
# ============================
# [Simple FG Mask] Color + Edge only (No texture, No morphology)
# ============================

import numpy as np, cv2

# ---- 파라미터 (가볍게 시작) ----
BORDER_PX      = 13     # 테두리 폭(px) - 배경 표본 추출용
COLOR_SPACE    = "LAB"  # "LAB" 권장 (HSV도 가능)
COLOR_WEIGHT   = 1.6    # 색 점수 가중치 (↑ 전경을 색차 위주로)
EDGE_WEIGHT    = 2.75   # 에지 점수 가중치 (↑ 윤곽 위주로)
GAUSS_KSIZE    = 5      # 사전 블러(홀수)
CC_CONNECT     = 8      # 연결성(4/8)
AREA_MIN_RATIO = 0.03  # 연결요소 최소 면적(이미지 대비) — 잡점 제거
AREA_MAX_RATIO = 0.45    # 연결요소 최대 면적 — 배경 덩어리 컷
OTSU_BIAS      = 0.00   # Otsu 전에 fused에서 빼는 바이어스(0~0.2 시도)


def _to_colorspace(bgr, mode="LAB"):
    mode = mode.upper()
    if mode == "LAB":
        return cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB).astype(np.float32)
    elif mode == "HSV":
        return cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV).astype(np.float32)
    else:
        raise ValueError("COLOR_SPACE must be 'LAB' or 'HSV'.")

def _border_stats(colimg, border_px):
    H, W = colimg.shape[:2]
    m = np.zeros((H, W), np.uint8)
    m[:border_px,:] = 1; m[-border_px:,:] = 1; m[:,:border_px] = 1; m[:,-border_px:] = 1
    samples = colimg[m>0].reshape(-1, colimg.shape[2])
    mu  = np.median(samples, axis=0).astype(np.float32)
    cov = np.cov(samples.T).astype(np.float32) if len(samples)>=10 else np.eye(colimg.shape[2], dtype=np.float32)
    return mu, cov

def _mahalanobis(X, mu, cov):
    C = cov.shape[0]
    inv = np.linalg.inv(cov + 1e-6*np.eye(C, dtype=np.float32))
    d = (X - mu).astype(np.float32)
    return np.sqrt(np.einsum('...i,ij,...j->...', d, inv, d)).astype(np.float32)

def _grad_mag(gray):
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(gx, gy)
    mag = mag / (mag.max() + 1e-6)
    return mag.astype(np.float32)

def _norm01(x):
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    return np.zeros_like(x) if mx-mn < 1e-6 else (x-mn)/(mx-mn)

def make_fg_mask_simple(bgr):
    """
    텍스처/모폴로지 없이:
    - 테두리 배경 통계로 Mahalanobis 색 거리 + Sobel 에지 → 점수 결합 → Otsu
    - 작은/너무 큰 연결요소만 면적으로 필터링
    """
    H, W = bgr.shape[:2]
    blur = cv2.GaussianBlur(bgr, (GAUSS_KSIZE, GAUSS_KSIZE), 0)

    col = _to_colorspace(blur, COLOR_SPACE)
    mu, cov = _border_stats(col, BORDER_PX)

    dist = _mahalanobis(col.reshape(-1, col.shape[2]), mu, cov).reshape(H, W)
    color_score = _norm01(dist)

    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    edge_score = _grad_mag(gray)

    fused = COLOR_WEIGHT*color_score + EDGE_WEIGHT*edge_score
    fused = _norm01(fused)
    if OTSU_BIAS != 0.0:
        fused = np.clip(fused - OTSU_BIAS, 0, 1)

    fused_u8 = (fused*255).astype(np.uint8)
    _, mask = cv2.threshold(fused_u8, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # 형태학 없이 → 면적 기반 연결요소 필터만
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=int(CC_CONNECT))
    out = np.zeros_like(mask)
    img_area = H*W
    for i in range(1, num):
        x,y,w,h,area = stats[i]
        if area < AREA_MIN_RATIO*img_area or area > AREA_MAX_RATIO*img_area:
            continue
        out[labels==i] = 255
    return out

def propose_boxes_from_mask(
    mask,
    min_area_ratio=0.001,   # 이미지 대비 최소 면적(노이즈 컷)
    max_area_ratio=0.90,    # 이미지 대비 최대 면적(배경 큰 덩어리 컷)
    pad=4,                  # 박스 패딩(px)
    connectivity=8,         # 연결성(4 또는 8)
    merge=True,             # 겹친 박스 병합 여부
    merge_iou=0.30          # 병합 임계 IoU
):
    """
    전경 마스크(0/255) → 후보 박스 목록 [(x,y,w,h), ...] 생성
    - 연결요소 기반 바운딩 박스 추출
    - 면적 비율로 1차 필터
    - 패딩 부여 후 이미지 경계로 클립
    - (옵션) IoU 기준으로 박스 병합
    """
    if mask is None or mask.size == 0:
        return []

    # 마스크 이진화 보정(0/255 보장)
    if mask.dtype != np.uint8:
        binm = (mask > 0).astype(np.uint8)
    else:
        binm = (mask > 0).astype(np.uint8)

    H, W = binm.shape[:2]
    img_area = H * W
    if img_area == 0:
        return []

    num, labels, stats, _ = cv2.connectedComponentsWithStats(binm, connectivity=int(connectivity))
    boxes = []
    for i in range(1, num):  # 0은 배경
        x, y, w, h, area = stats[i]
        if area < min_area_ratio * img_area or area > max_area_ratio * img_area:
            continue

        # 패딩
        x0 = max(0, x - pad)
        y0 = max(0, y - pad)
        x1 = min(W - 1, x + w + pad - 1)
        y1 = min(H - 1, y + h + pad - 1)

        xx, yy = int(x0), int(y0)
        ww, hh = int(max(1, x1 - x0 + 1)), int(max(1, y1 - y0 + 1))
        boxes.append((xx, yy, ww, hh))

    if not merge or len(boxes) <= 1:
        boxes.sort(key=lambda b: b[2]*b[3], reverse=True)
        return boxes

    # ---------------------------
    # 간단 병합(높은 IoU 박스 합치기)
    # ---------------------------
    def _xywh_to_xyxy(b):
        x, y, w, h = b
        return (x, y, x + w - 1, y + h - 1)

    def _xyxy_to_xywh(b):
        x1, y1, x2, y2 = b
        return (x1, y1, x2 - x1 + 1, y2 - y1 + 1)

    def _iou(a, b):
        ax1, ay1, ax2, ay2 = a
        bx1, by1, bx2, by2 = b
        iw = max(0, min(ax2, bx2) - max(ax1, bx1) + 1)
        ih = max(0, min(ay2, by2) - max(ay1, by1) + 1)
        inter = iw * ih
        if inter <= 0:
            return 0.0
        area_a = (ax2 - ax1 + 1) * (ay2 - ay1 + 1)
        area_b = (bx2 - bx1 + 1) * (by2 - by1 + 1)
        return inter / float(area_a + area_b - inter + 1e-9)

    xyxy = [_xywh_to_xyxy(b) for b in boxes]
    changed = True
    while changed and len(xyxy) > 1:
        changed = False
        new_xyxy = []
        used = [False] * len(xyxy)

        for i in range(len(xyxy)):
            if used[i]:
                continue
            merged = xyxy[i]
            used[i] = True
            for j in range(i + 1, len(xyxy)):
                if used[j]:
                    continue
                if _iou(merged, xyxy[j]) >= merge_iou:
                    # 합집합(바운딩 박스)
                    x1 = min(merged[0], xyxy[j][0])
                    y1 = min(merged[1], xyxy[j][1])
                    x2 = max(merged[2], xyxy[j][2])
                    y2 = max(merged[3], xyxy[j][3])
                    merged = (x1, y1, x2, y2)
                    used[j] = True
                    changed = True
            new_xyxy.append(merged)
        xyxy = new_xyxy

    merged_boxes = [_xyxy_to_xywh(b) for b in xyxy]
    merged_boxes.sort(key=lambda b: b[2]*b[3], reverse=True)
    return merged_boxes


In [10]:
import joblib

MODEL_DIR = './saved_models'
CLASSIFIER_PATH = os.path.join(MODEL_DIR, 'classifier_K800_H576_S16_WGT.pkl')
VOCAB_PATH      = os.path.join(MODEL_DIR, 'vocab_K800_H576_S16_WGT.pkl')

classifier = joblib.load(CLASSIFIER_PATH)
vocab = joblib.load(VOCAB_PATH)

CLASS_NAMES = [
    "Ants","Bees","Beetles","Caterpillars","Earthworms","Earwigs",
    "Grasshoppers","Moths","Slugs","Snails","Wasps","Weevils"
]


In [11]:
def extract_dense_sift(img, step=16):
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kps = [cv2.KeyPoint(x, y, step)
           for y in range(0, gray.shape[0], step)
           for x in range(0, gray.shape[1], step)]
    _, desc = sift.compute(gray, kps)
    return desc

def create_bovw_histogram(desc, vocab):
    k = vocab.n_clusters
    hist = np.zeros(k, np.float32)
    if desc is not None:
        words = vocab.predict(desc)
        hist, _ = np.histogram(words, bins=np.arange(k+1))
        hist = hist / (np.linalg.norm(hist) + 1e-6)
    return hist

def get_color_histogram(img, bins=16):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    chans = cv2.split(hsv)
    hists = [cv2.calcHist([c],[0],None,[bins],[0,256]) for c in chans]
    for h in hists: cv2.normalize(h, h)
    return np.hstack([h.flatten() for h in hists])

def extract_combined_features(img, vocab, W_BOVW=1.0, W_HOG=0.5, W_COLOR=1.0):
    sift_desc = extract_dense_sift(img)
    bovw = create_bovw_histogram(sift_desc, vocab)
    h = cv2.HOGDescriptor((320,320), (128,128), (64,64), (64,64), 9)
    hog_feat = h.compute(cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (320,320))).flatten()
    color_feat = get_color_histogram(img, bins=16)
    return np.hstack([W_BOVW*bovw, W_HOG*hog_feat, W_COLOR*color_feat])


In [12]:
def detect_and_classify(img):
    mask = make_fg_mask_simple(img)
    boxes = propose_boxes_from_mask(mask)
    results = []

    for (x, y, w, h) in boxes:
        crop = img[y:y+h, x:x+w]
        if crop.size == 0: continue
        crop_resized = cv2.resize(crop, (320, 320))
        feat = extract_combined_features(crop_resized, vocab)
        pred = classifier.predict(feat.reshape(1, -1))[0]
        results.append((CLASS_NAMES[pred], (x,y,w,h)))
    return results


In [14]:
from tqdm import tqdm

def evaluate_full_valid_set_fixed(IMG_DIR, LBL_DIR, iou_thr=0.5):
    img_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith('.jpg')]
    total_gt = 0
    det_TP, det_FP, det_FN = 0, 0, 0
    cls_TP, cls_FP, cls_FN = 0, 0, 0

    for fname in tqdm(img_files, desc="🔍 Evaluating full valid set (fixed)"):
        img_path = os.path.join(IMG_DIR, fname)
        lbl_path = os.path.join(LBL_DIR, fname.replace('.jpg', '.txt'))

        img = cv2.imread(img_path)
        if img is None: continue
        H, W = img.shape[:2]
        gts = yolo_txt_to_xyxy(lbl_path, W, H)
        total_gt += len(gts)
        if len(gts) == 0:
            continue

        preds = detect_and_classify(img)
        pred_boxes = [((x, y, x+w, y+h), cname) for cname, (x,y,w,h) in preds]

        matched_gt = set()
        matched_pred = set()

        for i, (p_box, cname) in enumerate(pred_boxes):
            best_iou, best_j = 0, -1
            for j, (gt_cls, gt_box) in enumerate(gts):
                if j in matched_gt: continue
                iou = iou_xyxy(p_box, gt_box)
                if iou > best_iou:
                    best_iou, best_j = iou, j
            # 탐지 성공
            if best_iou >= iou_thr:
                det_TP += 1
                matched_gt.add(best_j)
                matched_pred.add(i)
                # 분류까지 맞은 경우
                if cname == CLASS_NAMES[gts[best_j][0]]:
                    cls_TP += 1
                else:
                    cls_FP += 1
            else:
                det_FP += 1

        det_FN += (len(gts) - len(matched_gt))

    # --- 계산 (안정성 체크) ---
    det_precision = det_TP / max(1, (det_TP + det_FP))
    det_recall = det_TP / max(1, (det_TP + det_FN))
    det_f1 = 2 * det_precision * det_recall / max(1e-9, (det_precision + det_recall))

    cls_precision = cls_TP / max(1, (cls_TP + cls_FP))
    cls_recall = cls_TP / max(1, total_gt)   # 분모는 전체 GT (탐지+분류 둘 다 성공해야 하므로)
    cls_f1 = 2 * cls_precision * cls_recall / max(1e-9, (cls_precision + cls_recall))

    print("\n==============================================")
    print("📊 Precision / Recall / F1 Breakdown (Full valid set, fixed)")
    print("----------------------------------------------")
    print("🟠 Detection Stage")
    print(f" Precision : {det_precision*100:.2f}%")
    print(f" Recall    : {det_recall*100:.2f}%")
    print(f" F1-score  : {det_f1*100:.2f}%")
    print("----------------------------------------------")
    print("🔵 Classification Stage")
    print(f" Precision : {cls_precision*100:.2f}%")
    print(f" Recall    : {cls_recall*100:.2f}%")
    print(f" F1-score  : {cls_f1*100:.2f}%")
    print("==============================================")

    return (det_precision, det_recall, det_f1,
            cls_precision, cls_recall, cls_f1)

# 🚀 실행
evaluate_full_valid_set_fixed(VAL_IMG_DIR, VAL_LBL_DIR)


🔍 Evaluating full valid set (fixed): 100%|█| 1095/1095 [01:05<00:00, 16.64it/s]


📊 Precision / Recall / F1 Breakdown (Full valid set, fixed)
----------------------------------------------
🟠 Detection Stage
 Precision : 55.50%
 Recall    : 48.92%
 F1-score  : 52.00%
----------------------------------------------
🔵 Classification Stage
 Precision : 52.59%
 Recall    : 25.73%
 F1-score  : 34.55%


(0.5549915397631133,
 0.4891871737509321,
 0.5200158541418946,
 0.5259146341463414,
 0.25727069351230425,
 0.34551827741612423)

In [15]:
from tqdm import tqdm

def evaluate_end_to_end(IMG_DIR, LBL_DIR, iou_thr=0.5):
    img_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith('.jpg')]
    total_gt, TP, FP, FN = 0, 0, 0, 0

    for fname in tqdm(img_files, desc="🚀 Evaluating End-to-End (Detection + Classification)"):
        img_path = os.path.join(IMG_DIR, fname)
        lbl_path = os.path.join(LBL_DIR, fname.replace('.jpg', '.txt'))

        img = cv2.imread(img_path)
        if img is None: continue
        H, W = img.shape[:2]
        gts = yolo_txt_to_xyxy(lbl_path, W, H)
        total_gt += len(gts)
        if len(gts) == 0:
            continue

        preds = detect_and_classify(img)
        pred_boxes = [((x, y, x+w, y+h), cname) for cname, (x,y,w,h) in preds]

        matched_gt = set()
        matched_pred = set()

        for i, (p_box, cname) in enumerate(pred_boxes):
            best_iou, best_j = 0, -1
            for j, (gt_cls, gt_box) in enumerate(gts):
                if j in matched_gt:
                    continue
                iou = iou_xyxy(p_box, gt_box)
                if iou > best_iou:
                    best_iou, best_j = iou, j

            if best_iou >= iou_thr:
                if cname == CLASS_NAMES[gts[best_j][0]]:
                    TP += 1
                else:
                    FP += 1  # 탐지는 맞았지만 분류는 틀림
                matched_gt.add(best_j)
                matched_pred.add(i)
            else:
                FP += 1  # 탐지 자체가 잘못됨

        FN += (len(gts) - len(matched_gt))

    # 최종 스코어 계산
    precision = TP / max(1, (TP + FP))
    recall = TP / max(1, (TP + FN))
    f1 = 2 * precision * recall / max(1e-9, precision + recall)

    print("\n==============================================")
    print("🏁 Final End-to-End Performance (Detection + Classification)")
    print("----------------------------------------------")
    print(f" Precision : {precision*100:.2f}%")
    print(f" Recall    : {recall*100:.2f}%")
    print(f" F1-score  : {f1*100:.2f}%")
    print("----------------------------------------------")
    print(f" GT objects: {total_gt}")
    print(f" True Positives : {TP}")
    print(f" False Positives: {FP}")
    print(f" False Negatives: {FN}")
    print("==============================================")

    return precision, recall, f1

# 🚀 실행
evaluate_end_to_end(VAL_IMG_DIR, VAL_LBL_DIR)


🚀 Evaluating End-to-End (Detection + Classification): 100%|█| 1095/1095 [01:03<


🏁 Final End-to-End Performance (Detection + Classification)
----------------------------------------------
 Precision : 29.19%
 Recall    : 33.50%
 F1-score  : 31.19%
----------------------------------------------
 GT objects: 1341
 True Positives : 345
 False Positives: 837
 False Negatives: 685


(0.2918781725888325, 0.33495145631067963, 0.3119349005424955)

In [16]:
import numpy as np
from tqdm import tqdm

def compute_ap(rec, prec):
    """
    Compute the average precision (AP) given recall and precision.
    Method follows the VOC 2010/COCO convention.
    """
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # Precision smoothing
    for i in range(len(mpre)-1, 0, -1):
        mpre[i-1] = np.maximum(mpre[i-1], mpre[i])

    i = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[i+1] - mrec[i]) * mpre[i+1])
    return ap


def evaluate_map(IMG_DIR, LBL_DIR, iou_thr=0.5):
    """
    Evaluate mAP@0.5 for detection + classification pipeline
    """
    img_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith('.jpg')]
    all_detections = {cls: [] for cls in CLASS_NAMES}
    all_annotations = {cls: 0 for cls in CLASS_NAMES}

    # ---- 1. Collect detections & annotations ----
    for fname in tqdm(img_files, desc="📈 Collecting detections for mAP"):
        img_path = os.path.join(IMG_DIR, fname)
        lbl_path = os.path.join(LBL_DIR, fname.replace('.jpg', '.txt'))
        img = cv2.imread(img_path)
        if img is None:
            continue
        H, W = img.shape[:2]
        gts = yolo_txt_to_xyxy(lbl_path, W, H)
        preds = detect_and_classify(img)

        detected_gt = []

        for cname, (x, y, w, h) in preds:
            pred_box = (x, y, x+w, y+h)
            conf = 1.0  # or use detector probability if available
            pred_cls = cname
            all_detections[pred_cls].append({'conf': conf, 'bbox': pred_box, 'fname': fname})

        for gt_cls, gt_box in gts:
            all_annotations[CLASS_NAMES[gt_cls]] += 1

    # ---- 2. Compute AP per class ----
    aps = []
    for cname in CLASS_NAMES:
        detections = sorted(all_detections[cname], key=lambda x: x['conf'], reverse=True)
        n_gt = all_annotations[cname]
        if n_gt == 0:
            continue

        TP = np.zeros(len(detections))
        FP = np.zeros(len(detections))
        gt_used = {}

        for i, det in enumerate(detections):
            fname = det['fname']
            lbl_path = os.path.join(LBL_DIR, fname.replace('.jpg', '.txt'))
            img = cv2.imread(os.path.join(IMG_DIR, fname))
            if img is None:
                continue
            H, W = img.shape[:2]
            gts = yolo_txt_to_xyxy(lbl_path, W, H)
            pred_box = det['bbox']

            best_iou, best_j = 0, -1
            for j, (gt_cls, gt_box) in enumerate(gts):
                if CLASS_NAMES[gt_cls] != cname:
                    continue
                iou = iou_xyxy(pred_box, gt_box)
                if iou > best_iou:
                    best_iou, best_j = iou, j

            if best_iou >= iou_thr and (fname, best_j) not in gt_used:
                TP[i] = 1
                gt_used[(fname, best_j)] = True
            else:
                FP[i] = 1

        TP_cum = np.cumsum(TP)
        FP_cum = np.cumsum(FP)
        rec = TP_cum / (n_gt + 1e-6)
        prec = TP_cum / np.maximum(TP_cum + FP_cum, 1e-6)
        ap = compute_ap(rec, prec)
        aps.append(ap)
        print(f"AP({cname}) = {ap*100:.2f}%")

    mAP = np.mean(aps) if len(aps) > 0 else 0.0
    print("\n==============================================")
    print(f"🏁 mAP@0.5 (Detection+Classification) : {mAP*100:.2f}%")
    print("==============================================")
    return mAP


In [17]:
evaluate_map(VAL_IMG_DIR, VAL_LBL_DIR, iou_thr=0.5)

📈 Collecting detections for mAP: 100%|█████| 1095/1095 [01:01<00:00, 17.76it/s]


AP(Ants) = 4.04%
AP(Bees) = 11.92%
AP(Beetles) = 4.73%
AP(Caterpillars) = 1.36%
AP(Earthworms) = 3.45%
AP(Earwigs) = 3.25%
AP(Grasshoppers) = 6.97%
AP(Moths) = 44.04%
AP(Slugs) = 3.00%
AP(Snails) = 17.27%
AP(Wasps) = 23.47%
AP(Weevils) = 28.69%

🏁 mAP@0.5 (Detection+Classification) : 12.68%


np.float64(0.1268271182872758)

# XXX

In [6]:
# ============================================================
# test2 후속 셀 (완전 신규): 
#   전경 마스크 + 연결요소 후보 → HOG(+색) + SVM(이진: 곤충/배경) → 
#   클래스-무시 평가(Precision/Recall/F1) + 클래스별 균형 양성 샘플링
# ------------------------------------------------------------
# ※ 전제: test2에서 아래 함수가 이미 정의되어 있어야 함
#    - FG 마스크: make_fg_mask_texture(bgr) 또는 make_fg_mask_simple(bgr)
#    - 후보박스: propose_boxes_from_mask(mask)   # (x,y,w,h) 리스트 반환
# ※ 경로/임포트: test2의 경로 설정(../data/...)과 import가 이미 실행된 상태
# ============================================================

import os, random, time
import numpy as np
import cv2
from tqdm import tqdm
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# -----------------------------
# [A] 파이프라인 전역 파라미터
# -----------------------------
# 데이터 샘플링(너무 크면 오래 걸리므로 상한)
MAX_TRAIN_IMAGES = 1000
MAX_VAL_IMAGES   = 500
RANDOM_SEED      = 42
random.seed(RANDOM_SEED); np.random.seed(RANDOM_SEED)

# HOG 윈도/파라미터 (고정 크기로 리사이즈 후 특징 추출)
WIN_H, WIN_W     = 128, 128        # HOG 입력 크기 (H,W)
ORIENTATIONS     = 12               # 방향 수
PIXELS_PER_CELL  = (8, 8)         # 셀 크기(px)
CELLS_PER_BLOCK  = (2, 2)          # 블록(셀) 수

# 색 히스토그램(HSV) 사용 (배경과의 색 차 분리에 도움)
USE_HSV_HIST     = True
H_BINS, S_BINS, V_BINS = 32, 16, 16

# 이진 라벨 기준 (GT와의 IoU로 양/음성 정의)
IOU_POS_THR = 0.7   # 후보가 GT와 이 이상 겹치면 양성(전경)
IOU_NEG_THR = 0.10   # 랜덤 창이 모든 GT와 이 이하이면 음성(배경)

# 이미지당 크롭 수/비율 제한(데이터 폭주 방지)
MAX_CROPS_PER_IMG = 12
NEG_POS_RATIO     = 2.0
MAX_NEG_TOTAL     = 30000

# 클래스 균형(양성 샘플링 단계에서만 class_id 사용)
MAX_POS_PER_CLASS = 500  # 각 클래스별 양성 크롭 최대 확보 수
MIN_POS_PER_CLASS = 200   # (리포트용) 이상적 최소 목표

# 분류/검출/평가
SVM_C              = 1.5
SVM_MAX_ITER       = 5000
DET_THRESH         = 0.12    # p(foreground) 컷오프
NMS_IOU_THRESHOLD  = 0.6    # NMS IoU(클래스 무시)
MAX_PROPOSALS_PER_IMAGE = 200
EVAL_IOU_THR       = 0.50    # 평가 매칭 IoU

# 전경 마스크 함수 선택: test2에서 정의된 함수명을 연결
# FG_MASK_FN = make_fg_mask_texture
FG_MASK_FN = make_fg_mask_simple   # 필요 시 위/아래 중 택1

# -----------------------------
# [B] IOU/NMS/라벨 파서 유틸
# -----------------------------
def xywh_to_xyxy(b):
    x,y,w,h = b
    return (x, y, x+w, y+h)

def clip_box_xywh(b, W, H):
    x,y,w,h = map(int, b)
    x = max(0, min(x, W-1)); y = max(0, min(y, H-1))
    w = max(1, min(w, W-x)); h = max(1, min(h, H-y))
    return (x,y,w,h)

def iou_xyxy(a, b):
    ax1,ay1,ax2,ay2 = a
    bx1,by1,bx2,by2 = b
    iw = max(0, min(ax2,bx2) - max(ax1,bx1) + 1)
    ih = max(0, min(ay2,by2) - max(ay1,by1) + 1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = (ax2-ax1+1)*(ay2-ay1+1); area_b = (bx2-bx1+1)*(by2-by1+1)
    return inter / max(1.0, (area_a + area_b - inter))

def nms_xyxy(boxes, scores, iou_thr=0.5):
    if len(boxes)==0: return [], []
    boxes = np.asarray(boxes, dtype=np.float32)
    scores= np.asarray(scores, dtype=np.float32)
    x1,y1,x2,y2 = boxes.T
    areas = (x2-x1+1)*(y2-y1+1)
    order = scores.argsort()[::-1]
    keep=[]
    while order.size>0:
        i=order[0]; keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2-xx1+1); h = np.maximum(0.0, yy2-yy1+1)
        inter = w*h
        iou = inter/(areas[i] + areas[order[1:]] - inter + 1e-9)
        order = order[1:][iou <= iou_thr]
    return boxes[keep].astype(int).tolist(), scores[keep].tolist()

def fallback_load_annotations(lbl_path, img_hw):
    """
    YOLO txt 포맷 -> [(cid, (x,y,w,h)), ...] (픽셀 좌표)
    * 여기서는 'cid'는 학습 라벨로 쓰지 않지만, '양성 샘플 균형'에만 사용
    """
    H, W = img_hw
    out=[]
    if not os.path.exists(lbl_path):
        return out
    with open(lbl_path, 'r') as f:
        for line in f:
            s=line.strip().split()
            if len(s)!=5: continue
            cid, xc, yc, ww, hh = int(s[0]), float(s[1]), float(s[2]), float(s[3]), float(s[4])
            x = int((xc-ww/2)*W); y = int((yc-hh/2)*H)
            w = int(ww*W); h = int(hh*H)
            out.append((cid, (x,y,w,h)))
    return out

# -----------------------------
# [C] 특징 추출 (HOG + HSV 히스토)
# -----------------------------
def _resize_crop(bgr, box_xywh, out_hw):
    H,W = bgr.shape[:2]
    x,y,w,h = clip_box_xywh(box_xywh, W, H)
    crop = bgr[y:y+h, x:x+w]
    if crop.size==0: return None
    return cv2.resize(crop, (out_hw[1], out_hw[0]), interpolation=cv2.INTER_LINEAR)

def _hsv_hist(bgr, h_bins=32, s_bins=16, v_bins=16):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    histH = cv2.calcHist([hsv],[0],None,[h_bins],[0,180]).flatten()
    histS = cv2.calcHist([hsv],[1],None,[s_bins],[0,256]).flatten()
    histV = cv2.calcHist([hsv],[2],None,[v_bins],[0,256]).flatten()
    hist  = np.concatenate([histH, histS, histV]).astype(np.float32)
    return (hist / (np.sum(hist)+1e-6))

def extract_features(bgr_crop):
    """
    - 그레이스케일 HOG (모양/윤곽)
    - (옵션) HSV 히스토그램 (색상 분리)
    """
    gray = cv2.cvtColor(bgr_crop, cv2.COLOR_BGR2GRAY)
    fd = hog(gray, orientations=ORIENTATIONS, pixels_per_cell=PIXELS_PER_CELL,
             cells_per_block=CELLS_PER_BLOCK, block_norm='L2-Hys',
             visualize=False, transform_sqrt=True, feature_vector=True).astype(np.float32)
    if USE_HSV_HIST:
        hist = _hsv_hist(bgr_crop, H_BINS, S_BINS, V_BINS)
        fd = np.concatenate([fd, hist]).astype(np.float32)
    return fd

# -----------------------------
# [D] 경로 유틸/샘플링
# -----------------------------
def _sample_image_paths(img_dir, max_n, seed=42):
    paths = [os.path.join(img_dir, f) for f in os.listdir(img_dir)
             if f.lower().endswith(('.jpg','.jpeg','.png','.bmp'))]
    random.Random(seed).shuffle(paths)
    return paths[:max_n]

# -----------------------------
# [E] (핵심) 이진 학습 크롭 생성 - "클래스 균형 양성"
# -----------------------------
def build_binary_crops_from_dir_balanced(
    IMG_DIR, LBL_DIR,
    max_images=MAX_TRAIN_IMAGES,
    iou_pos_thr=IOU_POS_THR, iou_neg_thr=IOU_NEG_THR,
    max_crops_per_img=MAX_CROPS_PER_IMG,
    neg_pos_ratio=NEG_POS_RATIO,
    max_neg_total=MAX_NEG_TOTAL,
    max_pos_per_class=MAX_POS_PER_CLASS,
    min_pos_per_class=MIN_POS_PER_CLASS,
    seed=RANDOM_SEED
):
    """
    최종 라벨은 이진(0=배경, 1=전경/곤충).
    단, '양성 샘플'을 뽑을 때만 class_id를 사용해 클래스별로 고르게(상한/최소 목표) 확보.
    - 양성: (a) GT 박스 (b) 전경 후보 중 GT와 IoU>=iou_pos_thr
    - 음성: 랜덤 창 중 모든 GT와 IoU<=iou_neg_thr
    """
    random.seed(seed); np.random.seed(seed)
    img_paths = _sample_image_paths(IMG_DIR, max_images, seed=seed)
    if len(img_paths)==0: 
        raise FileNotFoundError(f"No images in {IMG_DIR}")

    # 클래스 집합 스캔
    per_class_count = {}
    per_class_quota = {}
    class_seen = set()
    for p in img_paths:
        bgr = cv2.imread(p); 
        if bgr is None: continue
        H,W = bgr.shape[:2]
        lbl = os.path.join(LBL_DIR, os.path.splitext(os.path.basename(p))[0]+'.txt')
        gts = fallback_load_annotations(lbl, (H,W))
        for cid,_ in gts: class_seen.add(cid)
    for cid in sorted(class_seen):
        per_class_count[cid] = 0
        per_class_quota[cid] = max_pos_per_class

    pos_crops, pos_labels = [], []
    neg_crops, neg_labels = [], []
    total_neg = 0

    for p in tqdm(img_paths, desc="[Balanced] Build binary crops"):
        bgr = cv2.imread(p); 
        if bgr is None: continue
        H,W = bgr.shape[:2]
        img_id = os.path.splitext(os.path.basename(p))[0]
        lbl   = os.path.join(LBL_DIR, img_id + '.txt')
        gts   = fallback_load_annotations(lbl, (H,W))  # [(cid,(x,y,w,h)),...]

        # 전경 후보(보강용)
        mask  = FG_MASK_FN(bgr)
        props = propose_boxes_from_mask(mask) or []

        added_pos = 0

        # (a) GT 자체를 클래스 균형으로 양성 확보
        for cid, (x,y,w,h) in gts:
            if per_class_count.get(cid,0) >= per_class_quota.get(cid,0): 
                continue
            crop = _resize_crop(bgr, (x,y,w,h), (WIN_H, WIN_W))
            if crop is None: continue
            pos_crops.append(crop); pos_labels.append(1)
            per_class_count[cid] += 1; added_pos += 1
            if added_pos >= max_crops_per_img: break

        # (b) 후보 중 GT와 잘 겹치는 것(같은 클래스 quota 남아있을 때만) 보강
        if added_pos < max_crops_per_img and len(props)>0 and len(gts)>0:
            gt_xyxy = [(cid, xywh_to_xyxy(bb)) for cid,bb in gts]
            for (px,py,pw,ph) in props:
                if added_pos >= max_crops_per_img: break
                p_xyxy = xywh_to_xyxy((px,py,pw,ph))
                # quota 남은 클래스 중 가장 IoU 큰 GT 찾기
                best_iou, best_cid = 0.0, None
                for cid, g in gt_xyxy:
                    if per_class_count[cid] >= per_class_quota[cid]:
                        continue
                    iou = iou_xyxy(p_xyxy, g)
                    if iou > best_iou:
                        best_iou, best_cid = iou, cid
                if best_cid is not None and best_iou >= iou_pos_thr:
                    crop = _resize_crop(bgr, (px,py,pw,ph), (WIN_H, WIN_W))
                    if crop is None: continue
                    pos_crops.append(crop); pos_labels.append(1)
                    per_class_count[best_cid] += 1; added_pos += 1

        # (c) 음성: 이미지당 음성/양성 비율 제한
        want_neg = min(int(neg_pos_ratio*added_pos), max_crops_per_img - added_pos)
        trials = 0
        gt_boxes_xyxy = [xywh_to_xyxy(bb) for _,bb in gts]
        while want_neg > 0 and total_neg < max_neg_total and trials < 60:
            trials += 1
            rw = np.random.randint(int(0.6*WIN_W), int(1.4*WIN_W))
            rh = np.random.randint(int(0.6*WIN_H), int(1.4*WIN_H))
            rx = np.random.randint(0, max(1, W - rw))
            ry = np.random.randint(0, max(1, H - rh))
            r_xyxy = (rx, ry, rx+rw, ry+rh)
            ok=True
            for g in gt_boxes_xyxy:
                if iou_xyxy(r_xyxy, g) > iou_neg_thr:
                    ok=False; break
            if not ok: continue
            crop = _resize_crop(bgr, (rx,ry,rw,rh), (WIN_H, WIN_W))
            if crop is None: continue
            neg_crops.append(crop); neg_labels.append(0)
            total_neg += 1; want_neg -= 1

    # (리포트) 클래스별 양성 확보 현황
    lacking = {cid: cnt for cid,cnt in per_class_count.items() if cnt < min_pos_per_class}
    print("\n[클래스별 양성 확보 현황]")
    for cid in sorted(per_class_count.keys()):
        mark = "(!)" if cid in lacking else "   "
        print(f"  class {cid:2d} : {per_class_count[cid]:5d} / {max_pos_per_class:5d} {mark}")
    if len(lacking)>0:
        print("  -> 일부 클래스는 최소 목표 미달(데이터 분포 한계 가능).")

    crops  = pos_crops + neg_crops
    labels = pos_labels + neg_labels
    print(f"[INFO] 양성(전경) {len(pos_labels):,} / 음성(배경) {len(neg_labels):,} / 총 {len(labels):,}")
    return crops, labels, per_class_count

# -----------------------------
# [F] 특징행렬 생성 & 학습
# -----------------------------
def feats_from_crops(crops):
    X=[]
    for c in tqdm(crops, desc="Extract feats"):
        X.append(extract_features(c))
    return np.asarray(X, dtype=np.float32)

# Train/Val 이진 크롭 생성 (클래스 균형)
train_crops, train_labels, pos_per_class = build_binary_crops_from_dir_balanced(
    TRAIN_IMG_DIR, TRAIN_LBL_DIR,
    max_images=MAX_TRAIN_IMAGES,
    iou_pos_thr=IOU_POS_THR, iou_neg_thr=IOU_NEG_THR,
    max_crops_per_img=MAX_CROPS_PER_IMG,
    neg_pos_ratio=NEG_POS_RATIO,
    max_neg_total=MAX_NEG_TOTAL,
    max_pos_per_class=MAX_POS_PER_CLASS,
    min_pos_per_class=MIN_POS_PER_CLASS
)

val_crops, val_labels, _ = build_binary_crops_from_dir_balanced(
    VAL_IMG_DIR, VAL_LBL_DIR,
    max_images=MAX_VAL_IMAGES,
    iou_pos_thr=IOU_POS_THR, iou_neg_thr=IOU_NEG_THR,
    max_crops_per_img=MAX_CROPS_PER_IMG,
    neg_pos_ratio=NEG_POS_RATIO,
    max_neg_total=MAX_NEG_TOTAL,
    max_pos_per_class=MAX_POS_PER_CLASS//2,  # 검증은 다소 작게
    min_pos_per_class=0
)

# 특징화
X_train = feats_from_crops(train_crops)
y_train = np.asarray(train_labels, dtype=np.int64)
X_val   = feats_from_crops(val_crops)
y_val   = np.asarray(val_labels, dtype=np.int64)

# 표준화 + SVM 이진 학습 + 확률보정
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)

svm_base = LinearSVC(C=SVM_C, max_iter=SVM_MAX_ITER, class_weight='balanced', dual=True)
t0=time.time(); svm_base.fit(X_train_sc, y_train); t1=time.time()
print(f"[SVM] 학습 완료: {t1-t0:.2f}s, classes={svm_base.classes_}")

clf = CalibratedClassifierCV(svm_base, cv=3, method='sigmoid')
t0=time.time(); clf.fit(X_train_sc, y_train); t1=time.time()
print(f"[Calib] 보정 완료: {t1-t0:.2f}s")

# -----------------------------
# [G] 이진 디텍터(전경 후보만 → 곤충/배경 판정)
# -----------------------------
def detect_binary(bgr, det_thr=DET_THRESH, nms_iou=NMS_IOU_THRESHOLD, max_props=MAX_PROPOSALS_PER_IMAGE):
    """
    반환: list[(score, (x1,y1,x2,y2)), ...]  # score= p(foreground)
    """
    mask  = FG_MASK_FN(bgr)
    props = propose_boxes_from_mask(mask) or []
    preds=[]
    for (x,y,w,h) in props[:max_props]:
        crop = _resize_crop(bgr, (x,y,w,h), (WIN_H, WIN_W))
        if crop is None: continue
        f   = extract_features(crop).reshape(1,-1)
        fsc = scaler.transform(f)
        prob = clf.predict_proba(fsc)[0]  # [p(bg), p(fg)]
        p_fg = float(prob[1])
        if p_fg >= det_thr:
            preds.append((p_fg, xywh_to_xyxy((x,y,w,h))))
    if len(preds)==0: return []
    boxes  = [bb for _,bb in preds]
    scores = [sc for sc,_ in preds]
    b_n, s_n = nms_xyxy(boxes, scores, iou_thr=nms_iou)
    return list(zip(s_n, b_n))

# [H] 클래스-무시 성능 평가(Precision/Recall/F1)
# -----------------------------
def evaluate_binary_detector_on_dir(IMG_DIR, LBL_DIR, max_imgs=MAX_VAL_IMAGES, iou_thr=EVAL_IOU_THR, verbose_every=25):
    paths = _sample_image_paths(IMG_DIR, max_imgs, seed=RANDOM_SEED+7)
    TP=0; FP=0; FN=0; total_gt=0; total_pred=0

    for i,p in enumerate(paths,1):
        bgr = cv2.imread(p)
        if bgr is None: continue
        H,W = bgr.shape[:2]
        lbl = os.path.join(LBL_DIR, os.path.splitext(os.path.basename(p))[0]+'.txt')
        gts = fallback_load_annotations(lbl, (H,W))
        gt_boxes = [xywh_to_xyxy(bb) for _,bb in gts]
        total_gt += len(gt_boxes)

        preds = detect_binary(bgr, det_thr=DET_THRESH, nms_iou=NMS_IOU_THRESHOLD)
        pred_boxes = [bb for _,bb in preds]
        total_pred += len(pred_boxes)

        used=set()
        for score, bb in preds:   # 고득점 먼저 들어옴
            best_iou, best_j = 0.0, -1
            for j,g in enumerate(gt_boxes):
                if j in used: continue
                iou = iou_xyxy(bb, g)
                if iou > best_iou:
                    best_iou, best_j = iou, j
            if best_iou >= iou_thr:
                TP += 1; used.add(best_j)
            else:
                FP += 1
        FN += (len(gt_boxes) - len(used))

        if verbose_every and (i%verbose_every==0):
            print(f"[{i}/{len(paths)}] 누적 GT:{total_gt} Pred:{total_pred}  TP:{TP} FP:{FP} FN:{FN}")

    prec = TP / max(1, TP+FP)
    rec  = TP / max(1, TP+FN)
    f1   = 2*prec*rec / max(1e-9, (prec+rec))
    print("\n========== [Binary Eval @ IoU=%.2f] =========="%iou_thr)
    print(f"Ground Truth Objects : {total_gt}")
    print(f"Total Predictions    : {total_pred}")
    print(f"True Positives (TP)  : {TP}")
    print(f"False Positives (FP) : {FP}")
    print(f"False Negatives (FN) : {FN}")
    print("---------------------------------------------")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("=============================================\n")




# -----------------------------
# [I] 평가 실행 (valid 먼저, 없으면 train로 갈아타도 됨)
# -----------------------------
if os.path.isdir(VAL_IMG_DIR) and len(os.listdir(VAL_IMG_DIR))>0:
    evaluate_binary_detector_on_dir(VAL_IMG_DIR, VAL_LBL_DIR, max_imgs=MAX_VAL_IMAGES, iou_thr=EVAL_IOU_THR)
else:
    print("[경고] valid 이미지가 비어 있음. train으로 평가 대체.")
    evaluate_binary_detector_on_dir(TRAIN_IMG_DIR, TRAIN_LBL_DIR, max_imgs=MAX_VAL_IMAGES, iou_thr=EVAL_IOU_THR)



[Balanced] Build binary crops: 100%|████████| 1000/1000 [00:59<00:00, 16.71it/s]



[클래스별 양성 확보 현황]
  class  0 :   223 /   500    
  class  1 :   119 /   500 (!)
  class  2 :   112 /   500 (!)
  class  3 :   183 /   500 (!)
  class  4 :   105 /   500 (!)
  class  5 :   109 /   500 (!)
  class  6 :   120 /   500 (!)
  class  7 :   170 /   500 (!)
  class  8 :    86 /   500 (!)
  class  9 :   146 /   500 (!)
  class 10 :   118 /   500 (!)
  class 11 :    94 /   500 (!)
  -> 일부 클래스는 최소 목표 미달(데이터 분포 한계 가능).
[INFO] 양성(전경) 1,585 / 음성(배경) 2,771 / 총 4,356


[Balanced] Build binary crops: 100%|██████████| 500/500 [00:29<00:00, 16.71it/s]



[클래스별 양성 확보 현황]
  class  0 :   100 /   250    
  class  1 :    60 /   250    
  class  2 :    54 /   250    
  class  3 :    61 /   250    
  class  4 :    40 /   250    
  class  5 :    66 /   250    
  class  6 :    54 /   250    
  class  7 :    79 /   250    
  class  8 :    55 /   250    
  class  9 :    77 /   250    
  class 10 :    80 /   250    
  class 11 :    72 /   250    
[INFO] 양성(전경) 798 / 음성(배경) 1,473 / 총 2,271


Extract feats: 100%|███████████████████████| 2271/2271 [00:12<00:00, 179.98it/s]


[SVM] 학습 완료: 88.34s, classes=[0 1]
[Calib] 보정 완료: 342.35s
[25/500] 누적 GT:29 Pred:23  TP:11 FP:12 FN:18
[50/500] 누적 GT:56 Pred:46  TP:21 FP:25 FN:35
[75/500] 누적 GT:88 Pred:73  TP:33 FP:40 FN:55
[100/500] 누적 GT:125 Pred:97  TP:50 FP:47 FN:75
[125/500] 누적 GT:173 Pred:121  TP:63 FP:58 FN:110
[150/500] 누적 GT:203 Pred:151  TP:81 FP:70 FN:122
[175/500] 누적 GT:232 Pred:178  TP:98 FP:80 FN:134
[200/500] 누적 GT:267 Pred:204  TP:115 FP:89 FN:152
[225/500] 누적 GT:293 Pred:227  TP:130 FP:97 FN:163
[250/500] 누적 GT:327 Pred:256  TP:148 FP:108 FN:179
[275/500] 누적 GT:363 Pred:278  TP:165 FP:113 FN:198
[300/500] 누적 GT:394 Pred:303  TP:182 FP:121 FN:212
[325/500] 누적 GT:427 Pred:331  TP:192 FP:139 FN:235
[350/500] 누적 GT:462 Pred:361  TP:205 FP:156 FN:257
[375/500] 누적 GT:491 Pred:386  TP:221 FP:165 FN:270
[400/500] 누적 GT:517 Pred:412  TP:236 FP:176 FN:281
[425/500] 누적 GT:543 Pred:441  TP:252 FP:189 FN:291
[450/500] 누적 GT:568 Pred:464  TP:267 FP:197 FN:301
[475/500] 누적 GT:594 Pred:490  TP:285 FP:205 FN:309
[50

In [7]:
import joblib, os

MODEL_DIR = './saved_models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Binary detector 모델 저장 (확률 보정된 버전)
joblib.dump(clf, os.path.join(MODEL_DIR, 'detector_binary_svm.pkl'))

# Scaler도 같이 저장해야 함 (검출 시 normalization 필요)
joblib.dump(scaler, os.path.join(MODEL_DIR, 'detector_scaler.pkl'))

print("✅ Detector model saved to ./saved_models/")


✅ Detector model saved to ./saved_models/
